In [2]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch import cuda
from torch.utils.data.dataloader import DataLoader

In [3]:
sys.path.append('../')
from mymodels.resnet_base_network import ResNet18

In [4]:
batch_size = 512
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

In [5]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [6]:
device = 'cpu'
# device = 'cuda' if cuda.is_available() else 'cpu'
# print(f"Training with: {device}")

train_dataset = datasets.STL10('/home/hong/dir1/PYTORCH-BYOL/', split='train', download=False,
                               transform=data_transforms)

test_dataset = datasets.STL10('/home/hong/dir1/PYTORCH-BYOL/', split='test', download=False,
                               transform=data_transforms)

In [7]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([3, 96, 96])


In [8]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True, pin_memory=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True, pin_memory=True)

In [9]:
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features

In [10]:
#duplication
encoder2 = ResNet18(**config['network'])
output_feature_dim2 = encoder2.projetion.net[0].in_features

In [11]:
#load pre-trained parameters
#load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Jul29_18-37-02_mango2/checkpoints/model.pth'),
#                         map_location=torch.device(torch.device(device)))
# load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Aug24_08-08-36_mango2/checkpoints/model.pth'),
#                          map_location=torch.device(torch.device(device)))
# epoch: 10
load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Sep26_15-10-29_mango2/checkpoints/model.pth'),
                        map_location=torch.device(torch.device(device)))

#load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Jul29_01-32-09_mango2/checkpoints/model.pth'),
#                         map_location=torch.device(torch.device(device)))\

# hong2
load_params2 = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Sep26_15-10-29_mango2/checkpoints/model.pth'),
                        map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


# hong
# duplication

load_params2 = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Sep26_15-10-29_mango2/checkpoints/model.pth'),
                        map_location=torch.device(torch.device(device)))

if '' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameteres for 2nd successfully loaded.")

# remove the projection head
encoder2 = torch.nn.Sequential(*list(encoder2.children())[:-1])    
encoder2 = encoder2.to(device)

In [12]:
encoder

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [13]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [14]:
class MyOne(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [15]:
logreg = LogisticRegression(output_feature_dim, 10)
logreg = logreg.to(device)

In [16]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [17]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

/home/hong/anaconda3/envs/cuda11/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training data shape: torch.Size([5000, 512]) torch.Size([5000])
Testing data shape: torch.Size([8000, 512]) torch.Size([8000])


In [18]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

In [19]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [20]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [21]:
optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10

# device = 'cuda' if cuda.is_available() else 'cpu'
# print(f"Training with: {device}")

for epoch in range(200):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = logreg(x)
        predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = logreg(x)
            predictions = torch.argmax(logits, dim=1)
            
            total += y.size(0)
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")

Testing accuracy: 60.5125
Testing accuracy: 68.225
Testing accuracy: 69.3625
Testing accuracy: 70.1875
Testing accuracy: 70.2875
Testing accuracy: 70.3875
Testing accuracy: 70.475
Testing accuracy: 70.2875
Testing accuracy: 70.4
Testing accuracy: 70.9
Testing accuracy: 70.6375
Testing accuracy: 70.8125
Testing accuracy: 70.5375
Testing accuracy: 70.7125
Testing accuracy: 70.525
Testing accuracy: 70.675
Testing accuracy: 70.425
Testing accuracy: 70.675
Testing accuracy: 70.7625
Testing accuracy: 70.675
